<a href="https://colab.research.google.com/github/SrijaG29/offc_dsa_prac/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Longest common prefix

In [ ]:
public class longestCommonPrefix {
    public static void main(String[] args) {
        String[] strs = {"flower","flow","flight"};
        if (strs.length == 0) {
            System.out.println("");
        }
        String result = strs[0];
        for (int i = 1; i < strs.length; i++) {
            String ans = "";
            int r = Math.min(result.length(), strs[i].length());
            for (int j = 0; j < r; j++) {
                if (result.charAt(j) == strs[i].charAt(j)) {
                    ans += result.charAt(j);
                } else {
                    break;
                }
            }
            result = ans;
            if (result.isEmpty()) {
                break;
            }
        }
        System.out.println(result);
    }
}

Html code

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Document</title>
    <link rel="stylesheet" href="style.css">
</head>
<body>
    <div class="navbar">
        <img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRqV8qU3m3W7cqHR9zkcdtry3bP-rQGybA-sw&s"
        alt="Logo" id="image">
        <h1 class="mainHeading">Expenses Calculator!</h1>
    </div>
    <div class="expensesForm">
        <h3 class="heading">Expenses Form</h3>
        <form class="formElements" onsubmit="getdetails(event)">
           <div class="select">
                <label for="expenseName">Expense Name: </label>
                <input type="text" id="expenseName" name="expenseName" required>
           </div>
           <div class="select">
                <label for="options">Select a category:</label>
                <select id="options" >
                    <option value="Select a Category">Select a Category</option>
                    <option value="Dressing">Dressing</option>
                    <option value="Food">Food</option>
                    <option value="Travel">Travel</option>
                    <option value="Other">Other</option>
                </select>
            </div>
            <div class="select" id="otherInputContainer"></div>
            <div class="select">
                <label for="totalAmount">Total Amount: </label>
                <input type="number" id="totalAmount" name="totalAmount" required>
            </div>
            <div class="select">
                <button type="submit">Submit</button>
            </div>
        </form>
    </div>
    <div class="additionalDetails">
        <div class="firstpart">
            <div class="filtering">
                <p>Filtering options:</p>
                <select id="filteroptions" >
                    <option value="Select a Category">Select a Category</option>
                    <option value="Dressing">Dressing</option>
                    <option value="Food">Food</option>
                    <option value="Travel">Travel</option>
                </select>
            </div>
            <div class="values">
                <div class="totalsalary">
                    <label for="totalsalary">Enter total salary:</label>
                    <input type="number" id="totalsalary" name="totalsalary" required>
                </div>
                <div class="expensedetails">
                    <tr>
                        <td><b>Expenses</b></td>
                        <td><b>Category</b></td>
                        <td><b>Total Amount</b></td>
                        <td><b>Remaining Salary</b></td>
                    </tr>
                </div>
            </div>
        </div>
        <div class="secondpart">
            <div class="categoryWiseExpenses">Expenses</div>
        </div>

    </div>
    <script src="actions.js"></script>
</body>
</html>

css code

In [ ]:
body{
    display: flex;
    flex-direction: column;
    background-color: rgb(205, 231, 234);
    margin: 0;
}

.navbar{
    height: 20%;
    display: flex;
    border-bottom: 2px solid black;
    /* padding: 1%; */
    justify-content: flex-start;
    align-items: center;
    background-color: rgb(6, 55, 70);
    color: azure;
}

#image{
    height: 50px;
    width: 100px;
    border-radius: 20%;
    padding: 0 2%;

}

.heading{
    margin: 0;
    padding-bottom: 1%;
}

.expensesForm{
    height: 30%;
    padding: 1%;
    border-bottom: 2px solid black;
}

.formElements{
    display: flex;
    justify-content: space-between;
}
.select{
    display: flex;
    /* justify-content: center; */
    /* justify-content: space-between; */
}


#otherinput{
    border-width: 1px;
    border-style: solid;
    border-color: light-dark(rgb(118, 118, 118)),
}

.additionalDetails{
    display: flex;
}

.firstpart{
    width: 50%
}

.filtering{
    display: flex;
    justify-content: center;
    padding: 5px;
    /* width: 50%; */
    border-right: 2px solid black;
    align-items: center;
}

.values{
    justify-items: center;
}

#filteroptions{
    width: 40%;
    height: 30px;
    margin-left: 10px;
}

.secondpart{
    width: 50%;
}


javascript

In [ ]:
// This is to take only aphabelts in expensename element
let expenseNameInput = document.getElementById("expenseName");
expenseNameInput.addEventListener("input", function(event) {
    // Remove any non-alphabet characters
    event.target.value = event.target.value.replace(/[^A-Za-z ]+/g, '');
});


let options = document.getElementById("options");
let newele;
let newlabel;
options.onchange = () => {
    const container = document.getElementById("otherInputContainer");
    // Clear the container before adding new elements
    container.innerHTML = '';
    if (options.value === "Other") {
        // Create and append a label
        newlabel = document.createElement("label");
        newlabel.textContent = "Other Category:";
        container.appendChild(newlabel);
        // Create and append a textarea
        let newele = document.createElement("input");
        newele.type = "text";
        newele.setAttribute("id","otherinput")
        // This is to take only aphabelts in otherinput textarea element
        container.addEventListener("input", function(event) {
            // Remove any non-alphabet characters
        event.target.value = event.target.value.replace(/[^A-Za-z ]+/g, '');
        });
        container.appendChild(newele);
    }
};

let index = 0;
let details = {}
function getdetails(event){
    event.preventDefault()
    let category
    if(document.getElementById("options").value === "Other"){
        category = document.getElementById("otherinput").value
        // adding new category into options for future selection in form
        let optionselement = document.getElementById("options")
        let categoryoption = document.createElement("option")
        categoryoption.textContent = category
        categoryoption.value = category
        let optionExists = Array.from(optionselement.options).some(option => option.value === category);
        if (!optionExists) {
            // Insert the new option at the second-to-last position
            optionselement.insertBefore(categoryoption,optionselement.lastElementChild)
        }else {
            // If the option already exists, just set the dropdown value to the existing option
            optionselement.value = category;
        }


        // adding new category into options for future selection in filtering
        let optionsfilteringelement = document.getElementById("filteroptions")
        let categoryfilteringoption = document.createElement("option")
        categoryfilteringoption.textContent = category
        categoryfilteringoption.value = category
        let filteringOptionExists = Array.from(optionsfilteringelement.options).some(option => option.value === category);
        if (!filteringOptionExists) {
            // Insert the new option at the second-to-last position
            optionsfilteringelement.insertBefore(categoryfilteringoption,optionsfilteringelement.lastElementChild)
        }else {
            // If the option already exists, just set the dropdown value to the existing option
            optionsfilteringelement.value = category;
        }
    }
    else{
        category = document.getElementById("options").value
    }
    // console.log(category)
    let d = {
        expenseName: document.getElementById("expenseName").value,
        category: category,
        totalAmount: document.getElementById("totalAmount").value
    };
    details[index] = d;
    index = index + 1;
    console.log(details)
    // expensedetails()
}









